In [3]:
# ------------------------------------------------------------------------
#
# TITLE - fit_dens_all_accmask.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Fit density profiles to all Gaia/APOGEE halo data masked using the accreted 
region of the [Fe/H]-[Al/Fe] plane.
'''

__author__ = "James Lane"

In [4]:
### Imports

# Basic
import os, sys, copy, dill as pickle
import numpy as np

# Matplotlib and plotting 
import matplotlib.pyplot as plt

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import util as putil

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

A new version of galpy (1.9.0) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!
/home/lane/software/galpy/galpy/util/bovy_plot.py:7: FutureWarning: galpy.util.bovy_plot is being deprecated in favor of galpy.util.plot; functions inside of this module have also changed name, but all functions still exist; please switch to the new import and new function names, because the old import and function names will be removed in v1.9
  warnings.warn(



### Keywords, Pathing, Loading, Data Preparation

In [5]:
# %load ../../src/nb_modules/keywords_pathing_loading_data_prep.py
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'FEH_MIN_GSE','FEH_MAX_GSE','DF_VERSION','KSF_VERSION','NPROCS',
            'RO','VO','ZO']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,feh_min_gse,feh_max_gse,df_version,\
    ksf_version,nprocs,ro,vo,zo = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]
feh_range_gse = [feh_min_gse,feh_max_gse]
feh_range_all = [feh_min,feh_max_gse]
feh_range_fit = copy.deepcopy(feh_range_all) # Need to choose here

## Pathing
fit_paths = putil.prepare_paths(base_dir,apogee_dr,apogee_results_vers,gaia_dr,
                                df_version,ksf_version)
data_dir,version_dir,ga_dir,gap_dir,df_dir,ksf_dir,fit_dir = fit_paths

## Filenames
fit_filenames = putil.prepare_filenames(ga_dir,gap_dir,feh_range_gse)
apogee_SF_filename,apogee_effSF_filename,apogee_effSF_mask_filename,\
    iso_grid_filename,clean_kinematics_filename = fit_filenames

## File loading and data preparation
fit_stuff,other_stuff = putil.prepare_fitting(fit_filenames,
    [ndmod,dmod_min,dmod_max],ro,zo,return_other=True)
apogee_effSF_mask,dmap,iso_grid,jkmins,dmods,ds,effsel_grid,apof,\
    allstar_nomask,orbs_nomask = fit_stuff
Rgrid,phigrid,zgrid = effsel_grid
# apogee_SF,apogee_effSF_grid_inclArea,apogee_effSF_grid_inclArea_Jac = other_stuff

# ## Load the distribution functions
# df_filename = df_dir+'dfs.pkl'
# betas = [0.3,0.9]
# dfs = putil.load_distribution_functions(df_filename, betas)

# ## Load the APOGEE field information, can also similarly load 
# ## '...apogee_field_glons.npy', '...apogee_field_glats.npy', 
# ## '...apogee_field_location_ids.npy'
# apogee_fields = np.load(ga_dir+'apogee_fields.npy')

Loading config file from /home/lane/projects/ges-mass/notebooks/config.txt

Loading APOGEE sel. func. from /epsen_data/scr/lane/projects/ges-mass/data/gaia_apogee/apogee_dr16_l33_gaia_edr3/apogee_SF.dat
/home/lane/software/galpy/galpy/util/bovy_coords.py:7: FutureWarning: galpy.util.bovy_coords is being deprecated in favor of galpy.util.coords; all functions in there are the same; please switch to the new import, because the old import will be removed in v1.9
  warnings.warn(


Loading APOGEE eff. sel. func. from /epsen_data/scr/lane/projects/ges-mass/data/gaia_apogee/apogee_dr16_l33_gaia_edr3/apogee_effSF_grid_inclArea_-2.0_feh_-0.6.dat

Loading APOGEE eff. sel. func. mask from /epsen_data/scr/lane/projects/ges-mass/data/gaia_apogee/apogee_dr16_l33_gaia_edr3/apogee_effSF_grid_mask.npy

Loading cleaned kinematics from /epsen_data/scr/lane/projects/ges-mass/data/gaia_apogee_processed/apogee_dr16_l33_gaia_edr3/clean_kinematics_sampled.npy

Loading isochrone grid from /epsen_data/scr/lane

### Global Parameters

In [10]:
# %load ../../src/nb_modules/global_fitting_params.py
## general kwargs
verbose = True

## HaloFit kwargs (ordering follows HaloFit.__init__)
# allstar and orbs loaded in prep cell
init = None
init_type = 'ML'
# fit_type provided at runtime
mask_disk = True
mask_halo = False
mask_acc = True
# densfunc, selec provided at runtime
# effsel, effsel_grid, effsel_mask, dmods loaded in prep cell
nwalkers = 50
nit = int(5e3)
ncut = int(1e3)
# usr_log_prior provided at runtime
n_mass = 5000 # int(nwalkers*(nit-ncut))
int_r_range = [2.,70.]
iso = None # Will read from iso_grid_filename
# iso_filename, jkmins loaded in prep cell
# feh_range provided at runtime
# logg_range loaded in config cell
# fit_dir, gap_dir, ksf_dir loaded in prep cell
# version provided at runtime
# ro, vo, zo loaded in config cell

hf_kwargs = {## HaloFit parameters
             'allstar':allstar_nomask,
             'orbs':orbs_nomask,
             'init':init,
             'init_type':init_type,
             # 'fit_type':fit_type, # provided at runtime
             'mask_disk':mask_disk,
             'mask_halo':mask_halo,
             'mask_acc':mask_acc,
             ## _HaloFit parameters
             # 'densfunc':densfunc, # provided at runtime
             # 'selec':selec, # provided at runtime
             'effsel':apof,
             'effsel_mask':apogee_effSF_mask,
             'effsel_grid':effsel_grid,
             'dmods':dmods,
             'nwalkers':nwalkers,
             'nit':nit,
             'ncut':ncut,
             # 'usr_log_prior':usr_log_prior, # provided at runtime
             'n_mass':n_mass,
             'int_r_range':int_r_range,
             'iso':iso,
             'iso_filename':iso_grid_filename,
             'jkmins':jkmins,
             # 'feh_range':feh_range, # provided at runtime
             'logg_range':logg_range,
             'fit_dir':fit_dir,
             'gap_dir':gap_dir,
             'ksf_dir':ksf_dir,
             # 'version':version, # provided at runtime
             'verbose':verbose,
             'ro':ro,
             'vo':vo,
             'zo':zo}

## pmass.fit() function kwargs
# nprocs set in config file
force_fit = True
mle_init = True
just_mle = False
return_walkers = True
optimizer_method = 'Powell'
mass_int_type = 'spherical_grid'
batch_masses = True
make_ml_aic_bic = True
calculate_masses = True
post_optimization = True
mcmc_diagnostic = True

fit_kwargs = {# 'nprocs':nprocs, # Normally given at runtime 
              'force_fit':force_fit,
              'mle_init':mle_init,
              'just_mle':just_mle,
              'return_walkers':return_walkers,
              'optimizer_method':optimizer_method,
              'mass_int_type':mass_int_type,
              'batch_masses':batch_masses,
              'make_ml_aic_bic':make_ml_aic_bic,
              'calculate_masses':calculate_masses,
              'post_optimization':post_optimization,
              'mcmc_diagnostic':mcmc_diagnostic,
              }

# Fits to all halo data. Use -3 < [Fe/H] < -0.6

In [16]:
densfunc = pdens.triaxial_single_angle_zvecpa
selec = None
fit_type = 'all'
version = 'accmask_50w_5e3n'

hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)
print(hf.n_star)

Using informed init
init_type: uninformed
densfunc_source: None
1115


### Triaxial Single power law

In [27]:
def usr_log_prior(densfunc,params):
    return 0

densfunc = pdens.triaxial_single_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_single_angle_zvecpa
Version is: accmask_50w_5e3n/
Using init_type: ML
init_type: uninformed
densfunc_source: None
Doing maximum likelihood to find starting point
MLE successful
MLE result: [2.28113659 0.72467365 0.47749186 0.35193887 0.99977088 0.55122902]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/5000 per walker
mean tau: 32.98140613878879
mean nit/tau: 15.160057090833362
min nit/tau: 13.66205492805163
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.36,0.55]
total max nit/tau 13.66205492805163
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:16:19.151458
---

sampled 1000/5000 per walker
mean tau: 49.50844303747192
mean nit/tau: 20.198575003522542
min nit/tau: 17.874564264107914
[min,max] delta tau: [-0.41143840883789423,-0.24126874699093476]
[min,max] acceptance fraction: [0.36,0.54]
total max nit/tau 17.874564264107914
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:14:26.841687
---

sampled

### Triaxial exponential cutoff

In [28]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_single_cutoff_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_single_cutoff_zvecpa
    if params[1] < 2.: return -np.inf
    if params[1] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_single_cutoff_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_cutoff_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_single_cutoff_zvecpa
Version is: accmask_50w_5e3n/
Using init_type: ML
init_type: ML
densfunc_source: triaxial_single_angle_zvecpa
source_fit_data_dir: /epsen_data/scr/lane/projects/ges-mass/data/fitting/data/all/-3.0_feh_-0.6/triaxial_single_angle_zvecpa/accmask_50w_5e3n/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [ 2.02060059 54.74431477  0.72656734  0.48793554  0.42332656  0.99984693
  0.55023602]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/5000 per walker
mean tau: 40.37409890589301
mean nit/tau: 12.384177320351785
min nit/tau: 8.649628245177965
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.35,0.47]
total max nit/tau 8.649628245177965
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:17:31.057083
---

sampled 1000/5000 per walker
mean tau: 64.22769771070737
mean nit/tau: 15.569606815180771
min nit/tau: 10.122751044732151
[min,max] delta tau: [-0.49851438775083523,-0.25

### Triaxial broken power law

In [29]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] < 2.: return -np.inf
    if params[2] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_broken_angle_zvecpa
Version is: accmask_50w_5e3n/
Using init_type: ML
init_type: ML
densfunc_source: triaxial_single_angle_zvecpa
source_fit_data_dir: /epsen_data/scr/lane/projects/ges-mass/data/fitting/data/all/-3.0_feh_-0.6/triaxial_single_angle_zvecpa/accmask_50w_5e3n/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [ 2.22535132  2.32381525 15.69884144  0.70620947  0.46403553  0.41371299
  0.99960719  0.54864017]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/5000 per walker
mean tau: 43.35539027972749
mean nit/tau: 11.532591375005904
min nit/tau: 9.035582582985503
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.24,0.46]
total max nit/tau 9.035582582985503
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:15:33.763385
---

sampled 1000/5000 per walker
mean tau: 79.90487880412863
mean nit/tau: 12.51488037984898
min nit/tau: 9.893724743063034
[min,max] delta tau: [-0.51379056341168

### Triaxial double broken power law

In [30]:
def usr_log_prior(densfunc,params):
    # return 0
    assert densfunc is pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_double_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] > 10.: return -np.inf
    if params[3] < 2.: return -np.inf
    if params[3] > 55.: return -np.inf
    if params[4] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_double_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_double_broken_angle_zvecpa
Version is: accmask_50w_5e3n/
Using init_type: ML
init_type: ML
densfunc_source: triaxial_broken_angle_zvecpa
source_fit_data_dir: /epsen_data/scr/lane/projects/ges-mass/data/fitting/data/all/-3.0_feh_-0.6/triaxial_broken_angle_zvecpa/accmask_50w_5e3n/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [4.32462582e-07 2.31838321e+00 2.31838813e+00 4.25934535e+00
 4.26015437e+00 7.29065792e-01 4.78319644e-01 4.24911743e-01
 9.99759562e-01 5.52037398e-01]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/5000 per walker
mean tau: 47.78855556649965
mean nit/tau: 10.462756073558875
min nit/tau: 8.486027715395121
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.16,0.4]
total max nit/tau 8.486027715395121
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:15:06.495076
---

sampled 1000/5000 per walker
mean tau: 85.5584321527003
mean nit/tau: 11.687918710516472
min nit/t